# Seasonal Polar Plots of TOTALDEMAND - Month and Season

Uses nsw_df.parquet as base file for data

In [106]:
import plotly.express as px

import pandas as pd
import numpy as np
from pandas.plotting import autocorrelation_plot
import os
import matplotlib.pyplot as plt
import functools
import seaborn as sns
from scipy.stats import zscore
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.graphics.tsaplots as tsa
import statsmodels.api as sm
import statsmodels.formula.api as smf
from config import CFG
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from utils import csv_to_parquet, normalize_columns, month_to_season, plot_correlation_heatmap
link = './../data/NSW/nsw_df.parquet'
df = pd.read_parquet(link)
df = df[['TOTALDEMAND']]
print(df.head(1))
print(df.shape)
print(df.index.min())
print(df.index.max())
print(df.index.dtype)

                     TOTALDEMAND
2010-01-02 00:30:00       7343.3
(196464, 1)
2010-01-02 00:30:00
2021-03-18 00:00:00
datetime64[ns]


In [107]:
df = df.groupby(pd.Grouper(freq='ME'), dropna=False).mean() #Monthly Aggregation
df['month'] = df.index.month

df['season'] = df['month'].map(lambda m: 'Summer' if m in [1, 2, 12] else ('Autumn' if m in [3, 4, 5] else ('Winter' if m in [6, 7, 8] else 'Spring'))).astype('category')

df['month'] = pd.to_datetime(df['month'], format='%m').dt.strftime('%b').astype('category')


In [108]:
df['year'] = df.index.year
df = df[df['year'] % 2 == 0] #Filter for even years only
df['year'] = df['year'].astype('category')

In [109]:
df.head(3)

,TOTALDEMAND,month,season,year
2010-01-31,8993.321418,Jan,Summer,2010
2010-02-28,9167.571458,Feb,Summer,2010
2010-03-31,8712.444422,Mar,Autumn,2010


In [110]:
fig = px.line_polar(df, r="TOTALDEMAND", theta="month", color="year", 
                    line_close=True,
                    color_discrete_sequence=px.colors.sequential.Plasma_r,
                    template="plotly_dark")
fig.update_layout(polar=dict(radialaxis=dict(tickfont=dict(size=10)))) #Decreases radial axis font size
fig.show()

In [111]:
fig = px.line_polar(df, r="TOTALDEMAND", theta="season", color="year", 
                    line_close=True,
                    color_discrete_sequence=px.colors.sequential.Plasma_r,
                    template="plotly_dark")
fig.update_layout(polar=dict(radialaxis=dict(tickfont=dict(size=10)))) #Decreases radial axis font size
fig.show()